In [1]:
from google.colab import files
files.upload()

Saving Mini Project.zip to Mini Project.zip


{'Mini Project.zip': b'PK\x03\x04\x14\x00\x08\x00\x08\x00(\x99\xb3P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x10\x00sutd.pngUX\x0c\x00\xe91\xce^[\xbe\xc3^\xf5\x01\x14\x00t\xba\x05T\x1c\xcd\xd3/LB\x80@\x90\x10Bp\x0f.\x8bC\x16M\xb0`\x8b\xbb,\x0e\x8b\xcb\xe2\x0b\x84\xe0\x1e\xdc%\xb8\xbb\xbb\x04wg!\xb8\xbb\xbb}<\x7fy\xbf{\xde{\xee\x9c\xb3{\xbakJz\xaa\xa7\xab~=\xd5A\xf2 \t\x0c4B4\x04\x04\x04\x0c\xc9\xef\xa2\x8a\x08\x08\xafB_\xdaeo\x91_\xfe\x8f\x9cvM\x10\x10\xde|\x86\x88\x88\xc8K\x8a\x88\x90\xcb\xdb\xdb\x98@,\x8d\x11\x10x\xb1\xfdT\x96m\x16\x94?\n\x06\xd9(f\x7f\x12"W\x97\x08\x08\xee.SQ\xd2j ",\xe9A\x96\x8c\xce9\x94\xfa\x94\xb3;\xc7\xf0y\x03\xc8\x13T@\x84\xb2\xb0[[(\xe7\xee\xcd\x83\xfc\xf0J\x18\x8e\x88\xf5\x9d\x9f\x02\x03\xd9\xc0\x1d~\xd5^-\xa9\x14\xc0P(\xf9\xdd}\xe7\x0f\x16\x05\xcd!\x92\xcaj\x93\x93\x01\xad\xd2\x1f\xed\xcb\xbb\xc1/\xcb_\xb6\xf9\xb6u\x17D\x05\xc0\x90\x9a$\x02\x95\xc0\xb8\xa2\xd2\xd4\xd1\xcf\xb1\xd1y\x9fb\xd2d\x83\'\xaa\xa6\xa4\x11\xa7\x8avA\xcc\x91\x83\xc3\x1fX:

In [2]:
!unzip -qn Mini\ Project.zip

In [3]:
!cd data && wget -q -nc https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
!cd data && unzip -qn wikitext-2-v1.zip

<img src="sutd.png" alt="drawing" style="width:300px;"/>

## <center>50.040 Natural Language Processing, Summer 2020<center>
<center>**Due 19 June 2020, 5pm** <center>
Mini Project

**Write your student ID and name**


### STUDNET ID: 1002675

### Name:Chia Yew Ken

### Students with whom you have discussed (if any):

# Introduction

Language models are very useful for a wide range of applications, e.g., speech recognition and machine translation. Consider a sentence consisting of words $x_1, x_2, …, x_m$, where $m$ is the length of the sentence, the goal of language modeling is to model the probability of the sentence, where $m \geq 1$, $x_i \in V $ and $V$ is the vocabulary of the corpus:
$$p(x_1, x_2, …, x_m)$$
In this project, we are going to explore both statistical language model and neural language model on the [Wikitext-2](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) datasets. Download wikitext-2 word-level data and put it under the ``data`` folder.

## Statistical  Language Model

A simple way is to view words as independent random variables (i.e., zero-th order Markovian assumption). The joint probability can be written as:
$$p(x_1, x_2, …, x_m)=\prod_{i=1}^m p(x_i)$$
However, this model ignores the word order information, to account for which, under the first-order Markovian assumption, the joint probability can be written as:
$$p(x_0, x_1, x_2, …, x_m)= \prod_{i=1}^{m}p(x_i \mid x_{i-1})$$
Under the second-order Markovian assumption, the joint probability can be written as:
$$p(x_{-1}, x_0, x_1, x_2, …, x_m)= \prod_{i=1}^{m}p(x_i \mid x_{i-2}, x_{i-1})$$
Similar to what we did in HMM, we will assume that $x_{-1}=START, x_0=START, x_m = STOP$ in this definition, where $START, STOP$ are special symbols referring to the start and the end of a sentence.







### Parameter estimation

Let's use $count(u)$ to denote the number of times the unigram $u$ appears in the corpus, use $count(v, u)$ to denote the number of times the bigram $v, u$ appears in the corpus, and $count(w, v, u)$ the times the trigram $w, v, u$ appears in the corpus, $u \in V \cup STOP$ and $w, v \in V \cup START$.

And the parameters of the unigram, bigram and trigram models can be obtained using maximum likelihood estimation (MLE).

- In the unigram model, the parameters can be estimated as: $$p(u) = \frac {count(u)}{c}$$, where $c$ is the total number of words in the corpus.
- In the bigram model, the parameters can be estimated as:
$$p(u \mid v) = \frac{count(v, u)}{count(v)}$$
- In the trigram model, the parameters can be estimated as:
$$p(u \mid w, v) = \frac{count(w, v, u)}{count(w, v)}$$




In [4]:
%%javascript
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

### Smoothing the parameters
Note, it is likely that many parameters of bigram and trigram models will be 0 because the relevant bigrams and trigrams involved do not appear in the corpus. If you don't have a way to handle these 0 probabilities, all the sentences that include such bigrams or trigrams will have probabilities of 0.

We'll use a Add-k Smoothing method to fix this problem, the smoothed parameter can be estimated as:
\begin{equation}
p_{add-k}(u)= \frac{count(u)+k}{c+k|V^*|}
\end{equation}
\begin{equation}
p_{add-k}(u \mid v)= \frac{count(v, u)+k}{count(v)+k|V^*|}
\end{equation}
\begin{equation}
p_{add-k}(u \mid w, v)= \frac{count(w, v, u)+k}{count(w, v)+k|V^*|}
\end{equation}

where $k \in (0, 1)$ is the parameter of this approach, and $|V^*|$ is the size of the vocabulary $V^*$,here $V^*= V \cup STOP$. One way to choose the value of $k$ is by
optimizing the perplexity of the development set, namely to choose the value that minimizes the perplexity.



### Perplexity

Given a test set $D^{\prime}$ consisting of sentences $X^{(1)}, X^{(2)}, …, X^{(|D^{\prime}|)}$, each sentence $X^{(j)}$ consists of words $x_1^{(j)}, x_2^{(j)},…,x_{n_j}^{(j)}$, we can measure the probability of each sentence $s_i$, and the quality of the language model would be the probability it assigns to the entire set of test sentences, namely:
\begin{equation} 
\prod_j^{D^{\prime}}p(X^{(j)})
\end{equation}
Let's define average log2 probability as:
\begin{equation} 
l=\frac{1}{c^{\prime}}\sum_{j=1}^{|D^{\prime}|}log_2p(X^{(j)})
\end{equation}
$c^{\prime}$ is the total number of words in the test set, $D^{\prime}$ is the number of sentences. And the perplexity is defined as:
\begin{equation} 
perplexity=2^{-l}
\end{equation}

The lower the perplexity, the better the language model.

In [5]:
from collections import Counter, namedtuple
import itertools
import numpy as np

In [6]:
with open('data/wikitext-2/wiki.train.tokens', 'r', encoding='utf8') as f:
    text = f.readlines()
    train_sents = [line.lower().strip('\n').split() for line in text]
    train_sents = [s for s in train_sents if len(s)>0 and s[0] != '=']

In [7]:
print(train_sents[1])

['the', 'game', 'began', 'development', 'in', '2010', ',', 'carrying', 'over', 'a', 'large', 'portion', 'of', 'the', 'work', 'done', 'on', 'valkyria', 'chronicles', 'ii', '.', 'while', 'it', 'retained', 'the', 'standard', 'features', 'of', 'the', 'series', ',', 'it', 'also', 'underwent', 'multiple', 'adjustments', ',', 'such', 'as', 'making', 'the', 'game', 'more', '<unk>', 'for', 'series', 'newcomers', '.', 'character', 'designer', '<unk>', 'honjou', 'and', 'composer', 'hitoshi', 'sakimoto', 'both', 'returned', 'from', 'previous', 'entries', ',', 'along', 'with', 'valkyria', 'chronicles', 'ii', 'director', 'takeshi', 'ozawa', '.', 'a', 'large', 'team', 'of', 'writers', 'handled', 'the', 'script', '.', 'the', 'game', "'s", 'opening', 'theme', 'was', 'sung', 'by', 'may', "'n", '.']


### Question 1 [code][written]
1. Implement the function **"compute_ngram"** that computes n-grams in the corpus.
 (Do not take the START and STOP symbols into consideration for now.) 
 For n=1,2,3, the number of unique n-grams should be **28910/577343/1344047**, respectively.
2. List 10 most frequent unigrams, bigrams and trigrams as well as their counts.(Hint: use the built-in function .most_common in Counter class)

In [8]:
def compute_ngram(sents, n):
    '''
    Compute n-grams that appear in "sents".
    param:
        sents: list[list[str]] --- list of list of word strings
        n: int --- "n" gram
    return:
        ngram_set: set{str} --- a set of n-grams (no duplicate elements)
        ngram_dict: dict{ngram: counts} --- a dictionary that maps each ngram to its number occurence in "sents";
        This dict contains the parameters of our ngram model. E.g. if n=2, ngram_dict={('a','b'):10, ('b','c'):13}
        
        You may need to use "Counter", "tuple" function here.
    '''
    ngram_set = None
    ngram_dict = None
    ### YOUR CODE HERE
    ngrams = []
    for tokens in sents:
        for i in range(0, len(tokens) - n + 1):
            window = tokens[i:i+n]
            ngrams.append(tuple(window))
    
    from collections import Counter
    counter = Counter(ngrams)
    ngram_set = set(counter.keys())
    ngram_dict = counter
    
    ### END OF YOUR CODE
    return ngram_set, ngram_dict

In [9]:
### ~28xxx
unigram_set, unigram_dict = compute_ngram(train_sents, 1)
print(len(unigram_set))

28910


In [10]:
### ~57xxxx
bigram_set, bigram_dict = compute_ngram(train_sents, 2)
print(len(bigram_set))

577343


In [11]:
### ~134xxxx
trigram_set, trigram_dict = compute_ngram(train_sents, 3)
print(len(trigram_set))

1344047


In [12]:
# List 10 most frequent unigrams, bigrams and trigrams as well as their counts.
for d in [unigram_dict, bigram_dict, trigram_dict]:
    print(d.most_common(10))
    print()

[(('the',), 130519), ((',',), 99763), (('.',), 73388), (('of',), 56743), (('<unk>',), 53951), (('and',), 49940), (('in',), 44876), (('to',), 39462), (('a',), 36140), (('"',), 28285)]

[(('of', 'the'), 17242), (('in', 'the'), 11778), ((',', 'and'), 11643), (('.', 'the'), 11274), ((',', 'the'), 8024), (('<unk>', ','), 7698), (('to', 'the'), 6009), (('on', 'the'), 4495), (('the', '<unk>'), 4389), (('and', 'the'), 4331)]

[((',', 'and', 'the'), 1393), ((',', '<unk>', ','), 950), (('<unk>', ',', '<unk>'), 901), (('one', 'of', 'the'), 866), (('<unk>', ',', 'and'), 819), (('.', 'however', ','), 775), (('<unk>', '<unk>', ','), 745), (('.', 'in', 'the'), 726), (('.', 'it', 'was'), 698), (('the', 'united', 'states'), 666)]



### Question 2 [code][written]
In this part, we take the START and STOP symbols into consideration. So we need to pad the **train_sents** as described in "Statistical Language Model" before we apply "compute_ngram" function. For example, given a sentence "I like NLP", in a bigram model, we need to pad it as "START I like NLP STOP", in a trigram model, we need to pad it as "START START I like NLP STOP".

1. Implement the ``pad_sents function``.
2. Pad ``train_sents``.
3. Apply ``compute_ngram`` function to these padded sents. 
4. Implement ``ngram_prob`` function. Compute the probability for each n-gram in the variable **ngrams** according to Eq.(1)(2)(3) in **"smoothing the parameters"** .List down the n-grams that have 0 probability. 



In [13]:
###############################################
ngrams = list()
with open(r'data/ngram.txt','r') as f:
    for line in f:
        ngrams.append(line.strip('\n').split())
print(ngrams)
###############################################

[['the', 'computer'], ['go', 'to'], ['have', 'had'], ['and', 'the'], ['can', 'sea'], ['a', 'number', 'of'], ['with', 'respect', 'to'], ['in', 'terms', 'of'], ['not', 'good', 'bad'], ['first', 'start', 'with']]


In [14]:
START = '<START>'
STOP = '</STOP>'
###################################
def pad_sents(sents, n):
    '''
    Pad the sents according to n.
    params:
        sents: list[list[str]] --- list of sentences.
        n: int --- specify the padding type, 1-gram, 2-gram, or 3-gram.
    return:
        padded_sents: list[list[str]] --- list of padded sentences.
    '''
    padded_sents = None
    ### YOUR CODE HERE
    front = [START] * (n - 1)
    back = [STOP]
    padded_sents = [front + tokens + back for tokens in sents]
    ### END OF YOUR CODE
    return padded_sents

In [15]:
uni_sents = pad_sents(train_sents, 1)
bi_sents = pad_sents(train_sents, 2)
tri_sents = pad_sents(train_sents, 3)

In [16]:
unigram_set, unigram_dict = compute_ngram(uni_sents, 1)
bigram_set, bigram_dict = compute_ngram(bi_sents, 2)
trigram_set, trigram_dict = compute_ngram(tri_sents, 3)

In [17]:
### (28xxx, 58xxxx, 136xxxx)
len(unigram_set),len(bigram_set),len(trigram_set)

(28911, 580825, 1363266)

In [18]:
### ~ 200xxxx; total number of words in wikitext-2.train
num_words = sum([v for _,v in unigram_dict.items()])
print(num_words)

2024702


In [19]:
def ngram_prob(ngram, num_words, unigram_dic, bigram_dic, trigram_dic):
    '''
    params:
        ngram: list[str] --- a list that represents n-gram
        num_words: int --- total number of words
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        prob: float --- probability of the "ngram"
    '''
    prob = None
    ### YOUR CODE HERE
    tup = tuple(ngram)
    n = len(tup)
    fn = {
        1: lambda x: unigram_dic[x] / num_words,
        2: lambda x: bigram_dic[x] / unigram_dic[x[:-1]],
        3: lambda x: trigram_dic[x] / bigram_dic[x[:-1]],
    }[n]
    prob = fn(tup)
    ### END OF YOUR CODE
    return prob

In [20]:
### ~9.96e-05
ngram_prob(ngrams[0], num_words,unigram_dict, bigram_dict, trigram_dict)

9.960235674499498e-05

In [21]:
### List down the n-grams that have 0 probability. 
print([
    ng for ng in ngrams 
    if ngram_prob(ng, num_words, unigram_dict, bigram_dict, trigram_dict) == 0
])

[['can', 'sea'], ['not', 'good', 'bad'], ['first', 'start', 'with']]


### Question 3 [code][written]

1. Implement ``smooth_ngram_prob`` function to estimate ngram probability with ``add-k`` smoothing technique. Compute the smoothed probabilities of each n-gram in the variable **"ngrams"** according to Eq.(1)(2)(3) in **"smoothing the parameters"** section.
2. Implement ``perplexity`` function to compute the perplexity of the corpus "**valid_sents**" according to the Equations (4),(5),(6) in **perplexity** section. The computation of $p(X^{(j)})$ depends on the n-gram model you choose. If you choose 2-gram model, then you need to calculate $p(X^{(j)})$ based on Eq.(2) in **smoothing the parameter** section. Hint: convert probability to log probability.
3. Try out different $k\in [0.1, 0.3, 0.5, 0.7, 0.9]$ and different n-gram model ($n=1,2,3$). Find the n-gram model and $k$ that gives the best perplexity on "**valid_sents**" (smaller is better).

In [22]:
with open('data/wikitext-2/wiki.valid.tokens', 'r', encoding='utf8') as f:
    text = f.readlines()
    valid_sents = [line.lower().strip('\n').split() for line in text]
    valid_sents = [s for s in valid_sents if len(s)>0 and s[0] != '=']

uni_valid_sents = pad_sents(valid_sents, 1)
bi_valid_sents = pad_sents(valid_sents, 2)
tri_valid_sents = pad_sents(valid_sents, 3)

In [23]:
def smooth_ngram_prob(ngram, k, num_words, unigram_dic, bigram_dic, trigram_dic):
    '''
    params:
        ngram: list[str] --- a list that represents n-gram
        k: float 
        num_words: int --- total number of words
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        s_prob: float --- probability of the "ngram"
    '''
    s_prob = 0
    V = len(unigram_dic) + 1 
    ### YOUR CODE HERE\、
    tup = tuple(ngram)
    n = len(tup)
    fn = {
        1: lambda x: (unigram_dic[x] + k) / (num_words + k*V),
        2: lambda x: (bigram_dic[x] + k) / (unigram_dic[x[:-1]] + k*V),
        3: lambda x: (trigram_dic[x] + k) / (bigram_dic[x[:-1]] + k*V),
    }[n]
    s_prob = fn(tup)
    ### END OF YOUR CODE
    return s_prob

In [24]:
### ~ 9.31e-05
smooth_ngram_prob(ngrams[0], 0.5, num_words, unigram_dict, bigram_dict, trigram_dict)

9.311950336264874e-05

In [25]:
def perplexity(n, k, num_words, valid_sents, unigram_dic, bigram_dic, trigram_dic):
    '''
    compute the perplexity of valid_sents
    params:
        n: int --- n-gram model you choose. 
        k: float --- smoothing parameter.
        num_words: int --- total number of words in the traning set.
        valid_sents: list[list[str]] --- list of sentences.
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        ppl: float --- perplexity of valid_sents
    '''
    ppl = None
    ### YOUR CODE HERE
    ngrams = []
    for tokens in valid_sents:
        for i in range(0, len(tokens) - n + 1):
            window = tokens[i:i+n]
            ngrams.append(tuple(window))

    losses = [
        smooth_ngram_prob(ng, k, num_words, unigram_dic, bigram_dic, trigram_dic) 
        # ngram_prob(ng, num_words, unigram_dic, bigram_dic, trigram_dic) 
        for ng in ngrams
    ]
    # loss = np.sum(np.log(losses)) / num_words
    loss = np.mean(np.log(losses))
    ppl = np.exp(-loss)
    ### END OF YOUR CODE
    return ppl

In [26]:
### ~ 840
perplexity(1, 0.1, num_words, uni_valid_sents, unigram_dict, bigram_dict, trigram_dict)

833.464705004929

In [27]:
n = [1,2,3]
k = [0.1, 0.3, 0.5, 0.7, 0.9]
### YOUR CODE HERE
ppl_best = 1e9
n_best = None
k_best = None
for n_value in n:
    for k_value in k:
        ppl = perplexity(n_value, k_value, num_words, uni_valid_sents, unigram_dict, bigram_dict, trigram_dict)
        print(dict(n_value=n_value, k_value=k_value, ppl=ppl))
        if ppl < ppl_best:
            ppl_best, n_best, k_best = ppl, n_value, k_value

print(dict(n_best=n_best, k_best=k_best, ppl_best=ppl_best))
### END OF YOUR CODE

{'n_value': 1, 'k_value': 0.1, 'ppl': 833.464705004929}
{'n_value': 1, 'k_value': 0.3, 'ppl': 833.865817488325}
{'n_value': 1, 'k_value': 0.5, 'ppl': 834.3114466705376}
{'n_value': 1, 'k_value': 0.7, 'ppl': 834.7976589099709}
{'n_value': 1, 'k_value': 0.9, 'ppl': 835.3210490687667}
{'n_value': 2, 'k_value': 0.1, 'ppl': 804.6918648337755}
{'n_value': 2, 'k_value': 0.3, 'ppl': 1152.8672381011695}
{'n_value': 2, 'k_value': 0.5, 'ppl': 1399.8530753304933}
{'n_value': 2, 'k_value': 0.7, 'ppl': 1604.0183827086553}
{'n_value': 2, 'k_value': 0.9, 'ppl': 1782.6628376645583}
{'n_value': 3, 'k_value': 0.1, 'ppl': 5805.930887540093}
{'n_value': 3, 'k_value': 0.3, 'ppl': 8124.104705953042}
{'n_value': 3, 'k_value': 0.5, 'ppl': 9530.813230188001}
{'n_value': 3, 'k_value': 0.7, 'ppl': 10569.446528522254}
{'n_value': 3, 'k_value': 0.9, 'ppl': 11397.840234405312}
{'n_best': 2, 'k_best': 0.1, 'ppl_best': 804.6918648337755}


### Question 4 [code]

Evaluate the perplexity of the test data **test_sents** based on the best n-gram model and $k$ you have found on the validation data (Q 3.3).

In [28]:
with open('data/wikitext-2/wiki.test.tokens', 'r', encoding='utf8') as f:
    text = f.readlines()
    test_sents = [line.lower().strip('\n').split() for line in text]
    test_sents = [s for s in test_sents if len(s)>0 and s[0] != '=']

uni_test_sents = pad_sents(test_sents, 1)
bi_test_sents = pad_sents(test_sents, 2)
tri_test_sents = pad_sents(test_sents, 3)

In [29]:
### YOUR CODE HERE
perplexity(2, 0.1, num_words,bi_test_sents, unigram_dict, bigram_dict, trigram_dict)
### END OF YOUR CODE

731.992244548119

## Neural Language Model (RNN)


<img src="LM.png" alt="drawing" style="width:500px;"/>

We will create a LSTM language model as shown in figure and train it on the Wikitext-2 dataset. 
The data generators (train\_iter, valid\_iter, test\_iter) have been provided. 
The word embeddings together with the parameters in the LSTM model will be learned from scratch.

[Pytorch](https://pytorch.org/tutorials/) and [torchtext](https://torchtext.readthedocs.io/en/latest/index.html#) are required in this part. Do not make any changes to the provided code unless you are requested to do so. 

### Question 5 [code]
- Implement the ``__init__`` function in ``LangModel`` class.
- Implement the ``forward`` function in ``LangModel`` class.
- Complete the training code in ``train`` function.
    Then complete the testing code in  ``test`` function and 
    compute the perplexity of the test data ``test_iter``. The test perplexity should be below 150.

In [30]:
import torchtext
import torch
import torch.nn.functional as F
from torchtext.datasets import WikiText2
from torch import nn, optim
from torchtext import data
from nltk import word_tokenize
import nltk
nltk.download('punkt')
torch.manual_seed(222)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [31]:
def tokenizer(text):
    '''Tokenize a string to words'''
    return word_tokenize(text)

START = '<START>'
STOP = '<STOP>'
#Load and split data into three parts
TEXT = data.Field(lower=True, tokenize=tokenizer, init_token=START, eos_token=STOP)
train, valid, test = WikiText2.splits(TEXT) 

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:00<00:00, 6.80MB/s]


extracting


In [32]:
#Build a vocabulary from the train dataset
TEXT.build_vocab(train)
print('Vocabulary size:', len(TEXT.vocab))

Vocabulary size: 28908


In [33]:
BATCH_SIZE = 64
# the length of a piece of text feeding to the RNN layer
BPTT_LEN = 32           
# train, validation, test data
train_iter, valid_iter, test_iter = data.BPTTIterator.splits((train, valid, test),
                                                                batch_size=BATCH_SIZE,
                                                                bptt_len=BPTT_LEN,
                                                                repeat=False)

In [34]:
#Generate a batch of train data
batch = next(iter(train_iter))
text, target = batch.text, batch.target
# print(batch.dataset[0].text[:32])
# print(text[0:3],target[:3])
print('Size of text tensor',text.size())
print('Size of target tensor',target.size())

Size of text tensor torch.Size([32, 64])
Size of target tensor torch.Size([32, 64])


In [35]:
class LangModel(nn.Module):
    def __init__(self, lang_config):
        super(LangModel, self).__init__()
        self.vocab_size = lang_config['vocab_size']
        self.emb_size = lang_config['emb_size']
        self.hidden_size = lang_config['hidden_size']
        self.num_layer = lang_config['num_layer']
        
        self.embedding = None
        self.rnn = None
        self.linear = None
        
        ### TODO: 
        ###    1. Initialize 'self.embedding' with nn.Embedding function and 2 variables we have initialized for you
        ###    2. Initialize 'self.rnn' with nn.LSTM function and 3 variables we have initialized for you
        ###    3. Initialize 'self.linear' with nn.Linear function and 2 variables we have initialized for you
        ### Reference:
        ###        https://pytorch.org/docs/stable/nn.html
        
        ### YOUR CODE HERE (3 lines)
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.rnn = nn.LSTM(self.emb_size, self.hidden_size, self.num_layer)
        self.linear = nn.Linear(self.hidden_size, self.vocab_size)
        ### END OF YOUR CODE
        
    def forward(self, batch_sents, hidden=None):
        '''
        params:
            batch_sents: torch.LongTensor of shape (sequence_len, batch_size)
        return:
            normalized_score: torch.FloatTensor of shape (sequence_len, batch_size, vocab_size)
        '''
        normalized_score = None
        hidden = hidden
        ### TODO:
        ###      1. Feed the batch_sents to self.embedding  
        ###      2. Feed the embeddings to self.rnn. Remember to pass "hidden" into self.rnn, even if it is None. But we will 
        ###         use "hidden" when implementing greedy search.
        ###      3. Apply linear transformation to the output of self.rnn
        ###      4. Apply 'F.log_softmax' to the output of linear transformation
        ###
        ### YOUR CODE HERE 
        x = self.embedding(batch_sents)
        x, hidden = self.rnn(x, hidden)
        x = self.linear(x)
        x = F.log_softmax(x, dim=-1)    
        
        normalized_score = x
        ### END OF YOUR CODE
        return normalized_score, hidden

In [36]:
def train(model, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs):
    for n in range(num_epochs):
        train_loss = 0
        target_num = 0
        model.train()
        for batch in train_iter:
            
            text, targets = batch.text.to(device), batch.target.to(device)
            loss = None
            
            ### we don't consider "hidden" here. So according to the default setting, "hidden" will be None
            ### YOU CODE HERE (~5 lines)
            optimizer.zero_grad()
            outputs, hidden = model(text)
            loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
            loss.backward()
            optimizer.step()
            ### END OF YOUR CODE
            ##########################################
            train_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)

        train_loss /= target_num

        # monitor the loss of all the predictions
        val_loss = 0
        target_num = 0
        model.eval()
        for batch in valid_iter:
            text, targets = batch.text.to(device), batch.target.to(device)
            
            prediction,_ = model(text)
            loss = criterion(prediction.view(-1, vocab_size), targets.view(-1))
            
            val_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)
        val_loss /= target_num

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(n+1, train_loss, val_loss))            

In [37]:
def test(model, vocab_size, criterion, test_iter):
    '''
    params: 
        model: LSTM model
        test_iter: test data
    return:
        ppl: perplexity 
    '''
    ppl = None
    test_loss = 0
    target_num = 0
    with torch.no_grad():
        for batch in test_iter:
            text, targets = batch.text.to(device), batch.target.to(device)

            prediction,_ = model(text)
            loss = criterion(prediction.view(-1, vocab_size), targets.view(-1))

            test_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)

        test_loss /= target_num
        
        ### Compute perplexity according to "test_loss"
        ### Hint: Consider how the loss is computed.
        ### YOUR CODE HERE(1 line)

        # test_loss is float, not torch.Tensor
        ppl = np.exp(test_loss)
        ### END OF YOUR CODE
        return ppl

In [38]:
num_epochs=10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(TEXT.vocab)

config = {'vocab_size':vocab_size,
         'emb_size':128,
         'hidden_size':128,
         'num_layer':1}

LM = LangModel(config)
LM = LM.to(device)

criterion = nn.NLLLoss(reduction='mean')
optimizer = optim.Adam(LM.parameters(), lr=1e-3, betas=(0.7, 0.99))

In [39]:
train(LM, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs)

Epoch: 1, Training Loss: 6.0691, Validation Loss: 5.1777
Epoch: 2, Training Loss: 5.4015, Validation Loss: 4.9643
Epoch: 3, Training Loss: 5.1293, Validation Loss: 4.8661
Epoch: 4, Training Loss: 4.9561, Validation Loss: 4.8139
Epoch: 5, Training Loss: 4.8310, Validation Loss: 4.7835
Epoch: 6, Training Loss: 4.7311, Validation Loss: 4.7640
Epoch: 7, Training Loss: 4.6476, Validation Loss: 4.7501
Epoch: 8, Training Loss: 4.5766, Validation Loss: 4.7423
Epoch: 9, Training Loss: 4.5150, Validation Loss: 4.7384
Epoch: 10, Training Loss: 4.4606, Validation Loss: 4.7392


In [40]:
# < 150
test(LM, vocab_size, criterion, test_iter)

99.15262130357378

### Question 6 [code]
When we use trained language model to generate a sentence given a start token, we can choose either ``greedy search`` or ``beam search``. 
<img src="greedy.png" alt="drawing" style="width:500px;"/>

As shown above, ``greedy search`` algorithm will pick the token which has the highest probability and feed it to the language model as input in the next time step. The model will generate ``max_len`` number of tokens at most.

- Implement ``word_greedy_search``
- **[optional]** Implement ``word_beam_search`` 

In [41]:
def word_greedy_search(model, start_token, max_len):
    '''
    param:
        model: nn.Module --- language model
        start_token: str --- e.g. 'he'
        max_len: int --- max number of tokens generated
    return:
        strings: list[str] --- list of tokens, e.g., ['he', 'was', 'a', 'member', 'of',...]
    '''
    model.eval()
    ID = TEXT.vocab.stoi[start_token]
    strings = [start_token]
    hidden = None
    
    ### You may find TEXT.vocab.itos useful.
    ### YOUR CODE HERE
    outputs = [torch.full(size=(1,), fill_value=ID, dtype=torch.long)]
    outputs[0] = outputs[0].to(device)

    with torch.no_grad():
        for _ in range(max_len):
            inputs = outputs[-1].unsqueeze(dim=0)
            logits, hidden = model(inputs, hidden)
            outputs.append(logits[-1,:,:].argmax(dim=-1))
    
    for o in outputs[1:]:
        i = o.item()
        s = TEXT.vocab.itos[i]
        if s == "<eos>":
            break
        else:
            strings.append(s)
    ### END OF YOUR CODE 
    return strings

In [42]:
# BeamNode = namedtuple('BeamNode', ['prev_node', 'prev_hidden', 'wordID', 'score', 'length'])
# LMNode = namedtuple('LMNode', ['sent', 'score'])

def word_beam_search(model, start_token, max_len, beam_size):
    pass

In [43]:
word_greedy_search(LM, 'he', 64)

['he', 'was', 'a', 'member', 'of', 'the', '<', 'unk', '>', '.']

In [44]:
word_beam_search(LM, 'he', 64, 1)

# char-level LM

### Question 7 [code]
- Implement ``char_tokenizer``
- Implement ``CharLangModel``, ``char_train``, ``char_test``
- Implement ``char_greedy_search``

In [45]:
def char_tokenizer(string):
    '''
    param:
        string: str --- e.g. "I love this assignment"
    return:
        char_list: list[str] --- e.g. ['I', 'l', 'o', 'v', 'e', ' ', 't', 'h', 'i', 's', ...]
    '''
    char_list = None
    ### YOUR CODE HERE
    return list(string)
    ### END OF YOUR CODE
    return char_list

In [46]:
test_str = 'test test test'
char_tokenizer(test_str)

['t', 'e', 's', 't', ' ', 't', 'e', 's', 't', ' ', 't', 'e', 's', 't']

In [47]:
CHAR_TEXT = data.Field(lower=True, tokenize=char_tokenizer ,init_token='<START>', eos_token='<STOP>')
ctrain, cvalid, ctest = WikiText2.splits(CHAR_TEXT)  

In [48]:
CHAR_TEXT.build_vocab(ctrain)
print('Vocabulary size:', len(CHAR_TEXT.vocab))

Vocabulary size: 247


In [49]:
BATCH_SIZE = 32
# the length of a piece of text feeding to the RNN layer
BPTT_LEN = 128        
# train, validation, test data
ctrain_iter, cvalid_iter, ctest_iter = data.BPTTIterator.splits((ctrain, cvalid, ctest),
                                                                batch_size=BATCH_SIZE,
                                                                bptt_len=BPTT_LEN,
                                                                repeat=False)

In [50]:
class CharLangModel(nn.Module):
    def __init__(self, lang_config):
        ### YOUR CODE HERE
        super().__init__()
        self.model = LangModel(lang_config)
        
    def forward(self, batch_sents, hidden=None):
        ### YOUR CODE HERE
        return self.model(batch_sents, hidden)

In [51]:
def char_train(model, train_iter, valid_iter, criterion, optimizer, vocab_size, num_epochs):
    ### YOUR CODE HERE
    return train(model, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs)

In [52]:
def char_test(model, vocab_size, test_iter, criterion):
    ### YOUR CODE HERE
    return test(model, vocab_size, criterion, test_iter)

In [53]:
num_epochs=10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
char_vocab_size = len(CHAR_TEXT.vocab)

config = {'vocab_size':char_vocab_size,
         'emb_size':128,
         'hidden_size':128,
         'num_layer':1}

CLM = CharLangModel(config)
CLM = CLM.to(device)

char_criterion = nn.NLLLoss(reduction='mean')
char_optimizer = optim.Adam(CLM.parameters(), lr=1e-3, betas=(0.7, 0.99))

In [54]:
char_train(CLM, ctrain_iter, cvalid_iter, char_criterion, char_optimizer, char_vocab_size, num_epochs)

Epoch: 1, Training Loss: 1.8419, Validation Loss: 1.5488
Epoch: 2, Training Loss: 1.5470, Validation Loss: 1.4423
Epoch: 3, Training Loss: 1.4730, Validation Loss: 1.3961
Epoch: 4, Training Loss: 1.4351, Validation Loss: 1.3709
Epoch: 5, Training Loss: 1.4113, Validation Loss: 1.3537
Epoch: 6, Training Loss: 1.3945, Validation Loss: 1.3416
Epoch: 7, Training Loss: 1.3816, Validation Loss: 1.3317
Epoch: 8, Training Loss: 1.3712, Validation Loss: 1.3234
Epoch: 9, Training Loss: 1.3628, Validation Loss: 1.3166
Epoch: 10, Training Loss: 1.3559, Validation Loss: 1.3110


In [55]:
# <10
char_test(CLM, char_vocab_size, ctest_iter, char_criterion)

3.6844402887921337

In [56]:
def char_greedy_search(model, start_token, max_len):
    '''
    param:
        model: nn.Module --- language model
        start_token: str --- e.g. 'h'
        max_len: int --- max number of tokens generated
    return:
        strings: list[str] --- list of tokens, e.g., ['h', 'e', ' ', 'i', 's',...]
    '''   
    model.eval()
    ID = CHAR_TEXT.vocab.stoi[start_token]
    strings = [start_token]
    hidden = None
    
    ### You may find CHAR_TEXT.vocab.itos useful.
    ### YOUR CODE HERE
    strings = word_greedy_search(model, start_token, max_len) 
    ### END OF YOUR CODE 
    return strings

In [57]:
TEXT = CHAR_TEXT
print("".join(char_greedy_search(CLM, "h", 64)))

he state , and the story , and the story , and the story , and th


### Requirements:
- This is an individual report.
- Complete the code using Python.
- List students with whom you have discussed if there are any.
- Follow the honor code strictly.

### Free GPU Resources
We suggest that you run neural language models on machines with GPU(s). Google provides the free online platform [Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb), a research tool for machine learning education and research. It’s a Jupyter notebook environment that requires no setup to use as common packages have been  pre-installed. Google users can have access to a Tesla T4 GPU (approximately 15G memory). Note that when you connect to a GPU-based VM runtime, you are given a maximum of 12 hours at a time on the VM.

It is convenient to upload local Jupyter Notebook files and data to Colab, please refer to the [tutorial](https://colab.research.google.com/notebooks/io.ipynb). 

In addition, Microsoft also provides the online platform [Azure Notebooks](https://notebooks.azure.com/help/introduction) for research of data science and machine learning, there are free trials for new users with credits.